In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
import os
#DefinindoavariáveldeambientedoJava
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64"
#DefinindoavariáveldeambientedoSpark
os.environ["SPARK_HOME"]="/content/spark-3.1.2-bin-hadoop2.7"
!pip install -q findspark

O sistema n�o pode encontrar o caminho especificado.
'wget' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.
tar: Error opening archive: Failed to open 'spark-3.1.2-bin-hadoop2.7.tgz'

[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import findspark

findspark.init()

In [ ]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[1]").appName('SparkByExamples.com').getOrCreate()

spark

In [ ]:
dataframe1 = spark.createDataFrame([
    ("Monitor LCD", 25000),
    ("TV LED 44", 35000),
    ("Palio", 21000)
],
["Produto", "Preço"])

dataframe1.show()

In [ ]:
from datetime import datetime, date

dataframe2 = spark.createDataFrame([
    (563214, "Maria", date(2000, 8, 9))
], ["CPF", "Nome", "Data-Nasc"])

dataframe2.show()

In [ ]:
from pyspark.sql.functions import *

new_dataf = dataframe2.withColumn("Nº Sorteio", round(rand()*10))

new_dataf.show()

In [ ]:
from pyspark.sql import SparkSession

sc=SparkSession.builder.master('local[*]').getOrCreate()
#Verificandoseasessãofoicriada
sc

In [ ]:
dados_seguranca = sc.read.csv(
    path = "./ES_segDEZ2020.csv",
    inferSchema = True,
    header = True,
    sep = ';',
    encoding = "UTF-8"
)

type(dados_seguranca)
dados_seguranca.show()

In [ ]:
from pyspark.sql.types import *
dados_seguranca1 = dados_seguranca.withColumn("IDADE", dados_seguranca[
"IDADE"].cast(IntegerType()))
for col in dados_seguranca1.dtypes:
  print(col[0]+" - "+col[1])

In [ ]:
from pyspark.sql.functions import *
rows = dados_seguranca.count()
print(f"Rows in Dataframe: {rows}")

In [ ]:
from pyspark.sql.functions import *
dados_seguranca.select(count(dados_seguranca.IDADE)).show()

In [ ]:
from pyspark.sql.functions import avg
dados_seguranca1.select(avg('IDADE')).collect()

# Desafio 2 - Tendo uma lista de 3 números de um sorteio (Ex: [1,6,8]) e um
Dataframe com o nome de pessoas e 3 números sorteados aleatórios (1-10)
para cada pessoa, elabore uma 5ª coluna com a verificação se a pessoa ganhou
o sorteio.

### Criando sessão pyspark

In [96]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import when
import random

spark = SparkSession.builder.master("local[1]").appName('SparkByExamples.com').getOrCreate()

### Criando DataFrame

In [97]:
dataframe_sorteio = spark.createDataFrame([
    ("Maria", 1, 6, 8),
    ("José", 3, 4, 6),
    ("João", 4, 1, 1),
], ["Nome", "N1", "N2", "N3"])


### Gerando números do sorteio e capturando números da tabela

In [113]:
numeros_sorteio = [random.randint(1, 10), random.randint(1, 10), random.randint(1, 10)]
numeros_escolhidos = dataframe_sorteio.select(["N1", "N2", "N3"]).rdd.map(lambda x: x[0:]).collect()

### Filtra Tabela aplicando regra para o vencedor do sorteio

In [114]:
dataframe_sorteio.withColumn("Resultado", 
                             when((dataframe_sorteio.N1 == numeros_sorteio[0]) & 
                                  (dataframe_sorteio.N2 == numeros_sorteio[1]) & 
                                  (dataframe_sorteio.N3 == numeros_sorteio[2]), "Ganhou")
                             .otherwise("Não Ganhou")).show()
print(f'Números sorteados: {numeros_sorteio}')

+-----+---+---+---+----------+
| Nome| N1| N2| N3| Resultado|
+-----+---+---+---+----------+
|Maria|  1|  6|  8|Não Ganhou|
| José|  3|  4|  6|Não Ganhou|
| João|  4|  1|  1|Não Ganhou|
+-----+---+---+---+----------+

Números sorteados: [7, 8, 9]
